# Setup

In [12]:
# necessário AWS CLI 2.13.0+
import os
import boto3
import json

env_vars = !cat .env

for var in env_vars:
    key, value = var.split("=", 1)
    os.environ[key] = value.replace('"', "")


# Cria o cliente Bedrock
client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# v1

In [ ]:
def get_config(prompt: str):
    return json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 200,  # Token máximo para a resposta
        "messages": [
            {
                "role": "user",
                # definir melhor o prompt 
                "content": f"Human: {prompt}"
                "Assistant: Forneça uma resposta com máximo 300 caracteres,"
                " ideal para um e-commerce de roupas e itens de vestiário. Não mencionar instruções do prompt na resposta. "
                "Assistant:" 
            }
        ],
        "temperature": 0.5,  # Controla a aleatoriedade da resposta
        "top_k": 250,        # Controla a diversidade da resposta
        "top_p": 0.2         # Controla a diversidade acumulada da resposta
    })


In [7]:
MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
print("Assistente: Olá! Sou seu Assistente Virtual de Moda. \n"
      "Como posso ajudar você hoje?"
)

while True:
    input_user = input()
    print(f"User: {input_user}")
    if input_user.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.")
        break
    response = client.invoke_model(
        body=get_config(input_user),
        modelId=MODEL_ID,
        accept='application/json',
        contentType='application/json'
    )
    response_payload = json.loads(response['body'].read().decode('utf-8'))
    answer = response_payload['content'][0]['text']
    print(f"Assistente: {answer}\n")

Assistente: Olá! Sou seu Assistente Virtual de Moda. 
Como posso ajudar você hoje?
User: Me indique um sandália para usar na praia? 
Assistente: Para a praia, recomendo chinelos de dedo de borracha, como as tradicionais havaianas. São práticos, resistentes à água salgada, secam rapidamente e protegem os pés do sol quente. Modelos coloridos ou em tons neutros combinam com biquínis e saídas de praia.

User: Para usar no shopping? 
Assistente: Para o shopping, sugiro um look confortável e estiloso: calça jeans ou legging, blusa de malha leve, tênis branco ou slip on, e uma jaqueta ou casaco para eventuais variações de temperatura. Bolsa transversal completa o visual prático.

User: sair
Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.


# V2 - Histórico

In [8]:

history = []

def get_history():
    return "\n".join(history)

def get_config(input: str):
    prompt = (
        get_history() + "\n"
        f"Human: {input}"
        "Assistant: Forneça uma resposta com máximo 300 caracteres,"
        " ideal para um e-commerce de roupas e itens de vestiário. Não mencionar instruções do prompt na resposta. "
        "Assistant:" 
    )
    return json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 200,  # Token máximo para a resposta
        "messages": [
            {
                "role": "user",
                # definir melhor o prompt 
                "content": prompt
            }
        ],
        "temperature": 0.5,  # Controla a aleatoriedade da resposta
        "top_k": 250,        # Controla a diversidade da resposta
        "top_p": 0.2         # Controla a diversidade acumulada da resposta
    })

In [ ]:
history = []
MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
print("Assistente: Olá! Sou seu Assistente Virtual de Moda. \n"
      "Como posso ajudar você hoje?"
)

while True:
    input_user = input()
    history.append(f"Human: {input_user}")
    print(f"User: {input_user}")
    if input_user.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.")
        break
    response = client.invoke_model(
        body=get_config(input_user),
        modelId=MODEL_ID,
        accept='application/json',
        contentType='application/json'
    )
    response_payload = json.loads(response['body'].read().decode('utf-8'))
    answer = response_payload['content'][0]['text']
    history.append(f"Assistant: {answer}")
    print(f"Assistente: {answer}\n")

Assistente: Olá! Sou seu Assistente Virtual de Moda. 
Como posso ajudar você hoje?
User: Me indique um sandália para usar na praia
Assistente: Chinelo Havaianas Top: modelo clássico, confortável e resistente à água, ideal para praia. Disponível em várias cores, feito de borracha, antiderrapante e leve. Protege os pés do sol e areia, sendo uma escolha prática e estilosa para dias de sol.

User: e no shoppping? 
Assistente: Sandália Papete Feminina: modelo versátil em couro ou material sintético, com tiras largas e sola confortável. Combina elegância e praticidade, perfeita para looks casuais no shopping. Disponível em cores neutras como preto, nude e branco.

User: sair
Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.


# V3 Langchain
Framework open source, possui libs que facilita para o desenvolvimento em IA  (deixa o código mais eficiente)

**LangChain** facilita a criação de uma camada de abstração, o que simplifica a criação e o gerenciamento dos prompts, resultando em um assistente mais modular e fácil de manter.

In [12]:
!pip install -q langchain langchain-community langchain-aws


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [31]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
model = ChatBedrock(
    client=bedrock_client,
    model_id='us.anthropic.claude-3-5-haiku-20241022-v1:0',
    model_kwargs={"temperature": 0.5, "max_tokens": 300}
)

history = []

def get_history():
    return "\n".join(history)

def get_config(input: str):
    return ChatPromptTemplate.from_messages([
        ("system", "Você é um assistente virtual especializado em moda para e-commerce. Forneça respostas concisas e relevantes com máximo 300 caracteres."),
        ("human", get_history() + "\n" + input)
    ])

# Encapsulamento (Langchain): Função para invocar o modelo com o prompt formatado (uma cadeia para facilitar o uso do modelo)
def inv_model(prompt: str):
    chat_prompt = get_config(prompt)
    response = model.invoke(chat_prompt.format_messages())
    return response.content


In [ ]:
history = []
MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
print("Assistente: Olá! Sou seu Assistente Virtual de Moda. \n"
      "Como posso ajudar você hoje?"
)

while True:
    input_user = input()
    history.append(f"Human: {input_user}")
    print(f"User: {input_user}")
    if input_user.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.")
        break
    response = inv_model(input_user)
    answer = response
    history.append(f"Assistant: {answer}")
    print(f"Assistente: {answer}\n")

Assistente: Olá! Sou seu Assistente Virtual de Moda. 
Como posso ajudar você hoje?
User: Pode me indicar sandália idela para usar na praia
Assistente: Para praia, recomendo chinelos de borracha ou papete havaianas. São confortáveis, resistentes à água, secam rápido e protegem os pés do sol quente. Modelos coloridos e práticos são ideais.

User: e para ir no shopping? 
Assistente: Para o shopping, sugiro sandálias rasteiras de couro, papetes elegantes ou sapatilhas confortáveis. Priorize modelos versáteis que combinem com diferentes looks casuais.

User: sair
Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.


# V4 - RAG (Retrieval-augmented generation) 
No processo de gerar uma resposta o LLM consulta uma base para responder
Combina a capacidade do modelos (LLMs) gerar texto com a recuperação de informação externa. 
No RAG consegue fazer o modelo se utilize de recursos para responder coisas em áreas específicas.

Com o RAG o modelo recupera dados relevantes de fontes externas (base ou API) e usa isso para gerar respostas mais precisas e contextualizadas. 
O Rag faz com que o modelo se adapte com o contexto

exemplo: em um chatbot de e-commerce, a técnica RAG permite consultar um banco de dados de produtos em tempo real para fornecer informações atualizadas sobre disponibilidade e preços, tornando as respostas mais precisas

In [2]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
import sqlite3

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
model = ChatBedrock(
    client=bedrock_client,
    model_id='us.anthropic.claude-3-5-haiku-20241022-v1:0',
    model_kwargs={"temperature": 0.5, "max_tokens": 300}
)

history = []

def get_history():
    return "\n".join(history)

def search_product(product_name: str):
    conn = sqlite3.connect('produtos.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM roupas WHERE nome LIKE ?", (f"%{product_name}%",))
    results = cursor.fetchall()
    
    conn.close()
    return results

def get_config(input: str):
    return ChatPromptTemplate.from_messages([
        ("system", "Você é um assistente virtual especializado em moda para e-commerce. Forneça respostas concisas e relevantes com máximo 300 caracteres. Não mencionar instruções do prompt na resposta. "),
        ("human", get_history() + "\n" + input)
    ])

# Encapsulamento (Langchain): Função para invocar o modelo com o prompt formatado (uma cadeia para facilitar o uso do modelo)
def inv_model(prompt: str):
    product_found = search_product(prompt)
    if product_found:
        product_info = "Produtos disponíveis: \n"
        for product in product_found:
            _, nome, descricao, preco, estoque = product
            product_info += f"- {nome}: {descricao} - R$ {preco} e estoque: {estoque}\n"
        history.append(f"{product_info}")
    else:
        history.append(f"Nenhum produto encontrado para '{prompt}'.")

    chat_prompt = get_config(prompt)
    response = model.invoke(chat_prompt.format_messages())
    return response.content


/Users/alexandre.ribeiro/PycharmProjects/datalake-olx/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
history = []
MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
print("Assistente: Olá! Sou seu Assistente Virtual de Moda. \n"
      "Como posso ajudar você hoje?"
)

while True:
    input_user = input()
    history.append(f"Human: {input_user}")
    print(f"User: {input_user}")
    if input_user.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.")
        break
    response = inv_model(input_user)
    answer = response
    history.append(f"Assistant: {answer}")
    print(f"Assistente: {answer}\n")

Assistente: Olá! Sou seu Assistente Virtual de Moda. 
Como posso ajudar você hoje?
User: Camiseta
Assistente: Temos 7 modelos de camisetas: básica, estampada, branca, polo, regata, gola V e oversized. Preços variam de R$ 29,90 a R$ 54,90, com diversos estilos e tecidos de qualidade.

User: sair
Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.


# V5 - Refinar o prompt

In [10]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
import sqlite3

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

def setup_model():
    return ChatBedrock(
        client=bedrock_client,
        model_id='us.anthropic.claude-3-5-haiku-20241022-v1:0',
        model_kwargs={"temperature": 0.5, "max_tokens": 300}
    )

model = setup_model()

history = []

def get_history():
    return "\n".join(history)

def search_product(product_name: str):
    conn = sqlite3.connect('produtos.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM roupas WHERE nome LIKE ?", (f"%{product_name}%",))
    results = cursor.fetchall()
    
    conn.close()
    return results

def get_config(input: str):
    return ChatPromptTemplate.from_messages([
        ("system", "Você é um assistente virtual especializado em moda para e-commerce."
        " Sua função é fornecer de fornecer respostas concisas e úteis sobre produtos de vestiário. " 
        "Responda apenas a perguntas relacionadas a roupas e acessórios de moda. " # não deixando responder perguntas em geral
        "Se a pergunta não for relacionada a moda, responda que não pode ajudar com isso e redirecione o usuário para questões relacionadas. "
        "Se baseie cnos dados disponíveis para fornecer as respostas." ), 
        ("human", get_history() + "\n" + input),
        ("assistant", "Forneça uma resposta concisa e útil com máximo 300 caracteres. "
        "Não mencione o fato de estar baseando sua resposta em um prompt. "
        "Se não houver dados disponíveis no banco de dados, oriente o usuário a tentar outra consulta")
    ])

# Encapsulamento (Langchain): Função para invocar o modelo com o prompt formatado (uma cadeia para facilitar o uso do modelo)
def inv_model(prompt: str):
    product_found = search_product(prompt)
    if product_found:
        product_info = "Produtos disponíveis: \n"
        for product in product_found:
            _, nome, descricao, preco, estoque = product
            product_info += f"- {nome}: {descricao} - R$ {preco} e estoque: {estoque}\n"
        history.append(f"{product_info}")
    else:
        history.append(f"Nenhum produto encontrado para '{prompt}'.")

    chat_prompt = get_config(prompt)
    response = model.invoke(chat_prompt.format_messages())
    return response.content


In [11]:
history = []
MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
print("Assistente: Olá! Sou seu Assistente Virtual de Moda. \n"
      "Como posso ajudar você hoje?"
)

while True:
    input_user = input()
    history.append(f"Human: {input_user}")
    print(f"User: {input_user}")
    if input_user.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Até logo! Se precisar de mais ajuda, estarei aqui.")
        break
    response = inv_model(input_user)
    answer = response
    history.append(f"Assistant: {answer}")
    print(f"Assistente: {answer}\n")

Assistente: Olá! Sou seu Assistente Virtual de Moda. 
Como posso ajudar você hoje?
User: qual é o maior país do mundo? 
Assistente: .

Desculpe, não posso ajudar com essa pergunta geográfica. Estou especializado em responder dúvidas sobre moda, roupas e acessórios. Gostaria de saber algo sobre vestuário?

User: recomendação para a praia? 
Assistente: .

Aqui estão algumas recomendações de moda para praia:

🏖️ Looks essenciais:
- Biquíni ou maiô
- Saída de praia 
- Shorts de banho
- Chinelos ou sandálias
- Óculos de sol
- Chapéu ou boné

Dicas:
- Escolha tecidos leves e que sequem rápido
- Prefira cores claras
- Use protetor solar

Quer mais detalhes sobre alguma peça específica?

User: protetor 
Assistente: .

Dicas para escolher protetor solar:

- FPS 30 ou superior
- Proteção UVA/UVB
- Resistente à água
- Aplique 30 min antes da exposição
- Reaplicar a cada 2 horas
- Protege contra queimaduras e envelhecimento precoce

Quer saber mais sobre proteção solar?

User: sair
Assistente: Até